In [ ]:
import pandas as pd
import numpy as np

#preprocess from raw data to 4Hz data

data = pd.read_csv(r'\new_SQM1.csv')
data['t'] = data['Time(s)']/0.25

time = 1
np_data = np.zeros([0,10])
for i in range(len(data)):
    if i%1000==0:
        print(i/len(data)*100)
    temp_data = data.loc[i]
    temp_t = temp_data['t']
    if abs(temp_t-time)<=(0.0417015*2):
        temp_data['t'] = time
        time += 1
        np_data = np.concatenate([np_data,temp_data.to_numpy().reshape(1,-1)])

ids = np.unique(data['VehicleID'].to_numpy())
np_data = np.zeros([0,10])
total_times = 0
for ID in ids:
    single_data = data[data['VehicleID']==ID].reset_index().drop(columns = ['index'])
    time = int(single_data.loc[0]['t'])+1
    for i in range(len(single_data)):
        if i%500==0:
            print(ID,(total_times+i)/len(data)*100)
        temp_data = single_data.loc[i]
        temp_t = temp_data['t']
        if abs(temp_t-time)<=(0.0417015*2):
            temp_data['t'] = time
            time += 1
            np_data = np.concatenate([np_data,temp_data.to_numpy().reshape(1,-1)])
    total_times += len(single_data)
    
new_pd = pd.DataFrame(np_data, columns = ['id','time','laneId','Lon','gap','v','Lat','length','Lat_v','t'])
new_pd.to_csv(r'\4Hz.csv',index=None)


In [ ]:
first_stage = new_pd
ids = np.unique(first_stage['id'].to_numpy())
np_data = np.zeros([0,12])
total_times = 0
for ID in ids:
    single_data = first_stage[first_stage['id']==ID].reset_index().drop(columns = ['index'])
    begin_time = single_data.loc[0]['t']
    if begin_time%2 != 0:
        for i in range(len(single_data)):
            if i%500==0:
                print(ID,(total_times+i)/len(data)*100)
            temp_data = single_data.loc[i]
            if single_data.loc[i]['t']%2==0:
                temp_data['lon v'] = (temp_data['v'] + single_data.loc[i-1]['v'])/2
                temp_data['lat v'] = (temp_data['Lat_v'] + single_data.loc[i-1]['Lat_v'])/2
                np_data = np.concatenate([np_data,temp_data.to_numpy().reshape(1,-1)])
    elif begin_time%2 == 0:
        for i in range(len(single_data)):
            if i%500==0:
                print(ID,(total_times+i)/len(data)*100)
            temp_data = single_data.loc[i]
            if single_data.loc[i]['t']%2==0 and i>0:
                temp_data['lon v'] = (temp_data['v'] + single_data.loc[i-1]['v'])/2
                temp_data['lat v'] = (temp_data['Lat_v'] + single_data.loc[i-1]['Lat_v'])/2
                np_data = np.concatenate([np_data,temp_data.to_numpy().reshape(1,-1)])
    total_times += len(single_data)

In [ ]:
second_stage = pd.DataFrame(np_data, columns = ['id','time','laneId','Lon','gap','v','Lat','length','Lat_v','t','lon v','lat v'])


In [ ]:
# Process 'code' data
import math

codes = []
for i in range(len(second_stage)):
    if i%500==0:
        print(i/len(second_stage)*100)
    
    lon_v = second_stage.loc[i]['lon v']
    lat_v = second_stage.loc[i]['lat v']
    tanh_lat_v = np.tanh(2*lat_v)
    
    if lon_v>20:
        code_lon = 6
    elif lon_v<20 and lon_v>0:
        code_lon = math.ceil(lon_v/4)
    elif lon_v == 0:
        code_lon = 1
    
    if tanh_lat_v<=-0.6:
        code_lat = 1
    elif tanh_lat_v>-0.6 and tanh_lat_v <= -0.2:
        code_lat = 2
    elif tanh_lat_v>-0.2 and tanh_lat_v<=0.2:
        code_lat = 3
    elif tanh_lat_v>0.2 and tanh_lat_v<=0.6:
        code_lat = 4
    elif tanh_lat_v>0.6:
        code_lat = 5
        
    code = (code_lon-1)*5+code_lat
    codes.append(code)

In [ ]:
second_stage['codes'] = pd.DataFrame(codes)
second_stage['t'] = second_stage['t']/2
second_stage.to_csv(r'\2Hz.csv',index=None)

In [ ]:
#second_stage = pd.read_csv(r'\2Hz.csv')

ids = np.unique(second_stage['id'].to_numpy())
np_data = np.zeros([0,15])
total_times = 0
for ID in ids:
    single_data = second_stage[second_stage['id']==ID].reset_index().drop(columns = ['index'])
    begin_time = single_data.loc[0]['t']
    if begin_time%2 != 0:
        for i in range(len(single_data)):
            if i%500==0:
                print(ID,(total_times+i)/len(second_stage)*100)
            temp_data = single_data.loc[i]
            if single_data.loc[i]['t']%2==0:
                temp_data['words'] = (single_data.loc[i-1]['codes']-1)*30+single_data.loc[i]['codes']
                temp_data['corpus'] = (int(single_data.loc[i-1]['codes']),int(single_data.loc[i]['codes']))
                np_data = np.concatenate([np_data,temp_data.to_numpy().reshape(1,-1)])
    elif begin_time%2 == 0:
        for i in range(len(single_data)):
            if i%500==0:
                print(ID,(total_times+i)/len(second_stage)*100)
            temp_data = single_data.loc[i]
            if single_data.loc[i]['t']%2==0 and i>0:
                temp_data['words'] = (single_data.loc[i-1]['codes']-1)*30+single_data.loc[i]['codes']
                temp_data['corpus'] = (int(single_data.loc[i-1]['codes']),int(single_data.loc[i]['codes']))
                np_data = np.concatenate([np_data,temp_data.to_numpy().reshape(1,-1)])
    total_times += len(single_data)

In [ ]:
third_stage = pd.DataFrame(np_data, columns = ['id','time','laneId','Lon','gap','v','Lat','length','Lat_v','t','lon v','lat v','codes','words','corpus'])
third_stage['t'] = third_stage['t']/2
third_stage.to_csv(r'\1Hz.csv',index=None)